## TTSvelte 0.0.1
### Convert site built with tailwind into a svelte app

#### steps
1. go to the site you want to convert
2. open the console
3. paste the code below
```js
function createTree(element, includeAttributes = false) {
  const tree = {
    tagName: element.tagName.toLowerCase(),
    children: [],
  };

  if (includeAttributes) {
    const attributes = Array.from(element.attributes).reduce((acc, attr) => {
      acc[attr.nodeName] = attr.nodeValue;
      return acc;
    }, {});

    tree.attributes = attributes;
  }

  Array.from(element.children).forEach(child => {
    tree.children.push(createTree(child, includeAttributes));
  });

  return tree;
}

// Usage:
const siteTree = createTree(document.body, true); // set the second argument to `false` to exclude attributes
console.log(JSON.stringify(siteTree, null, 2));

function saveFile(content, fileName) {
  const fileContent = new Blob([content], { type: 'application/json' });
  const downloadLink = document.createElement('a');
  
  downloadLink.href = URL.createObjectURL(fileContent);
  downloadLink.download = fileName;
  downloadLink.click();

  // Clean up to avoid memory leaks.
  setTimeout(() => {
    URL.revokeObjectURL(downloadLink.href);
    downloadLink.remove();
  }, 0);
}

// Usage:
const siteTree = createTree(document.body, true);
saveFile(JSON.stringify(siteTree, null, 2), 'site_tree.json');
```

4. copy the fille `site_tree.json` to the root of your project

run the following command to install the dependencies
```bash
pip install -r requirements.txt
```

5. run the TTsvelte compiler
```bash
ttsvelte -t site_tree.json -n <name of the app>
```


In [1]:
%pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

with open("site_tree.json", "r") as f:
    site_tree = json.load(f)

In [5]:
# import networkx as nx

# def build_graph(node, g, parent=None):
#     label = node["tagName"]
#     attributes = node.get("attributes", {})
#     id_attr = attributes.get("id", "")
#     class_attr = attributes.get("class", "")

#     if id_attr:
#         label += f"#{id_attr}"
#     if class_attr:
#         label += f".{class_attr.replace(' ', '.')}"

#     g.add_node(label)

#     if parent is not None:
#         g.add_edge(parent, label)

#     for child in node["children"]:
#         build_graph(child, g, label)

from anytree import Node

def build_anytree(node, parent=None):
    label = node["tagName"]
    attributes = node.get("attributes", {})
    id_attr = attributes.get("id", "")
    class_attr = attributes.get("class", "")

    if id_attr:
        label += f"#{id_attr}"
    if class_attr:
        label += f".{class_attr.replace(' ', '.')}"

    child_node = Node(label, parent=parent)

    for child in node["children"]:
        build_anytree(child, child_node)
        
    return child_node

In [7]:
# G = nx.DiGraph()
# build_graph(site_tree, G)

root = build_anytree(site_tree)


In [8]:
# import matplotlib.pyplot as plt

# pos = nx.kamada_kawai_layout(G)
# plt.figure(figsize=(12, 12))
# nx.draw(G, pos, with_labels=True, node_size=3000, font_size=10, font_weight="bold")
# plt.show()

from anytree import RenderTree

for pre, _, node in RenderTree(root):
    print(f"{pre}{node.name}")


body
├── div#__next
│   ├── script
│   ├── div.overflow-hidden.w-full.h-full.relative
│   │   ├── div.flex.h-full.flex-1.flex-col.md:pl-[260px]
│   │   │   └── main.relative.h-full.w-full.transition-width.flex.flex-col.overflow-hidden.items-stretch.flex-1
│   │   │       ├── div.flex-1.overflow-hidden
│   │   │       │   └── div.react-scroll-to-bottom--css-xsxul-79elbk.h-full.dark:bg-gray-800
│   │   │       │       └── div.react-scroll-to-bottom--css-xsxul-1n7m0yu
│   │   │       │           ├── div.px-2.py-10.relative.w-full.flex.flex-col.h-full
│   │   │       │           │   ├── div.flex.items-center.justify-center.gap-2
│   │   │       │           │   │   └── div.relative.w-full.md:w-1/2.lg:w-1/3.xl:w-1/4
│   │   │       │           │   │       └── button#headlessui-listbox-button-:r0:.relative.flex.w-full.cursor-default.flex-col.rounded-md.border.border-black/10.bg-white.py-2.pl-3.pr-10.text-left.focus:border-green-600.focus:outline-none.focus:ring-1.focus:ring-green-600.dark:bor